In [1]:
%matplotlib inline

# Bokeh for interactive, Seaborn for static

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

sns.set(style="whitegrid", palette="pastel")

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

from math import pi
import glob

In [2]:
def m_to_cm(x):
    return x * 100
def unit(x):
    return x
def scale_to_rads(x):
    return 20 * x

fsm_map = [
    {'name': 'Wheel Base'        , 'default': 0.08, 'minval': 0.08, 'maxval': 0.16, 'T': m_to_cm},
    {'name': 'Track Width'       , 'default': 0.12, 'minval': 0.08, 'maxval': 0.16, 'T': m_to_cm},
    {'name': 'Wheel Radius'      , 'default': 0.02, 'minval': 0.02, 'maxval': 0.03, 'T': m_to_cm},
    {'name': 'Weg Count'         , 'default': 3   , 'minval': 0   , 'maxval': 7.99, 'T': int  },
    {'name': 'Ext. Slope'        , 'default': 0.5 , 'minval': 0   , 'maxval': 1   , 'T': unit},
    {'name': 'Ext. Intercept'    , 'default': 0.5 , 'minval': 0   , 'maxval': 1   , 'T': unit},
    {'name': 'F. Speed'          , 'default': 1   , 'minval': 0   , 'maxval': 1   , 'T': scale_to_rads},
    {'name': 'F. Left Thresh'    , 'default': 0.17, 'minval': 0   , 'maxval': pi/2, 'T': np.rad2deg},
    {'name': 'F. Right Thresh'   , 'default':-0.17, 'minval':-pi/2, 'maxval': 0   , 'T': np.rad2deg},
    {'name': 'L. Speed Left'     , 'default':-1   , 'minval':-1   , 'maxval': 1   , 'T': scale_to_rads},
    {'name': 'L. Speed Right'    , 'default': 1   , 'minval':-1   , 'maxval': 1   , 'T': scale_to_rads},
    {'name': 'L. Forward Thresh' , 'default': 0.08, 'minval': 0   , 'maxval': pi/2, 'T': np.rad2deg},
    {'name': 'R. Speed Left'     , 'default': 1   , 'minval':-1   , 'maxval': 1   , 'T': scale_to_rads},
    {'name': 'R. Speed Right'    , 'default':-1   , 'minval':-1   , 'maxval': 1   , 'T': scale_to_rads},
    {'name': 'R. Forward Thresh' , 'default':-0.08, 'minval':-pi/2, 'maxval': 0   , 'T': np.rad2deg},
]
fsm_fancy_names = [
    r'WheelBase (cm)',r'TrackWidth (cm)',r'WheelRadius (cm)',r'StrutCount',r'$m_{ext}$ ($\%$)',
    r'$b_{ext}$ ($\%$)',r'F.Speed (rad $s^{-1}$)',r'F.ToLeftThresh ($^\circ$)',
    r'F.ToRightThresh ($^\circ$)',r'L.LeftSpeed (rad $s^{-1}$)',r'L.RightSpeed (rad $s^{-1}$)',
    r'L.ToForwardThresh ($^\circ$)',r'R.LeftSpeed (rad $s^{-1}$)',r'R.RightSpeed (rad $s^{-1}$)',
    r'R.ToForwardThresh ($^\circ$)'
]
for m, n in zip(fsm_map, fsm_fancy_names):
    m['fancy_name'] = n


bnn_map = [
    {'name': 'Wheel Base'        , 'default': 0.08, 'minval': 0.08, 'maxval': 0.16, 'T': m_to_cm},
    {'name': 'Track Width'       , 'default': 0.12, 'minval': 0.08, 'maxval': 0.16, 'T': m_to_cm},
    {'name': 'Wheel Radius'      , 'default': 0.02, 'minval': 0.02, 'maxval': 0.03, 'T': m_to_cm},
    {'name': 'Weg Count'         , 'default': 3   , 'minval': 0   , 'maxval': 7.99, 'T': int  },
    {'name': 'Act. Func.'        , 'default': 0   , 'minval': 0   , 'maxval': 2.99, 'T': int  },
]
bnn_map.extend([{
    'name': 'weight{}'.format(i), 
    'default': 0, 
    'minval': -4, 
    'maxval': 4, 
    'T': float} for i in range(12)])
bnn_fancy_names = [r'WheelBase (cm)',r'TrackWidth (cm)',r'WheelRadius (cm)',r'StrutCount',r'Act.Func.']
bnn_fancy_names += ['Weight {}'.format(widx) for widx in range(12)]
for m, n in zip(bnn_map, bnn_fancy_names):
    m['fancy_name'] = n


cmaes_header = ['iter', 'evals', 'sigma', '0', 'fitness']
fsm_header = cmaes_header + [arg['name'] for arg in fsm_map]
bnn_header = cmaes_header + [arg['name'] for arg in bnn_map]


def range_transform(x, a, b, c, d):
    return (x - a) * (d - c) / (b - a) + c

In [24]:

def get_data_from_files(glob_dir, header, param_map, exp_name):
    
    if exp_name == 'bnn':
        exp_name_full = 'ANN Direct'
    elif exp_name == 'fsm':
        exp_name_full = 'FSM'
    else:
        exp_name_full = 'ANN Twist'

    cols_to_keep = ['fitness', 'Wheel Base', 'Track Width', 'Wheel Radius', 'Weg Count']
    data = []

    for pop_filepath in sorted(glob.iglob(glob_dir, recursive=True)):
        
        data.append(pd.read_csv(pop_filepath, 
                                comment='%', 
                                header=None, 
                                names=header, 
                                sep=' ', 
                                usecols=cols_to_keep)
                   .assign(experiment=exp_name_full))

        data[-1]['fitness'] = (data[-1]['fitness'] * -1) + 1
        
        for m in param_map:
            if m['name'] not in cols_to_keep:
                continue
            data[-1][m['name']] = data[-1][m['name']].apply(
                lambda x: m['T'](range_transform(x, 0, 10, m['minval'], m['maxval'])))
    
    print('Found', len(data), 'data files in', glob_dir)

    df = pd.concat(data, ignore_index=True)
    return df


def get_data(num_obst, num_trials):
    exp_names = ['fsm', 'bnn', 'bnn_twist']
    exp_headers = [fsm_header, bnn_header, bnn_header]
    exp_maps = [fsm_map, bnn_map, bnn_map]
    data_glob_template = '../experiments/<exp>/seed*/outcmaesxrecentbest.dat'
    
    exp_data = []
    for exp_name, exp_header, exp_map in zip(exp_names, exp_headers, exp_maps):
        templ = data_glob_template
        exp_dirname = templ.replace('<exp>', f'{exp_name}-{num_obst}-{num_trials}')
        
        exp_data.append(get_data_from_files(exp_dirname, exp_header, exp_map, exp_name))

    df = pd.concat(exp_data, ignore_index=True, sort=False)
    return df

df = get_data(0, 1)
df = pd.melt(df, id_vars=['fitness', 'experiment'], var_name='parameter')
df.head()

Found 20 data files in ../experiments/fsm-0-1/seed*/outcmaesxrecentbest.dat
Found 20 data files in ../experiments/bnn-0-1/seed*/outcmaesxrecentbest.dat
Found 20 data files in ../experiments/bnn_twist-0-1/seed*/outcmaesxrecentbest.dat


,fitness,experiment,parameter,value
0,9.440000,FSM,Wheel Base,8.239525
1,9.520000,FSM,Wheel Base,9.230483
2,9.550000,FSM,Wheel Base,9.576996
3,9.543333,FSM,Wheel Base,8.477341
4,9.556667,FSM,Wheel Base,9.585825


In [51]:
# df_best = df.nlargest(240, 'fitness')
# df_best = df.groupby('experiment')['fitness'].nlargest(5)

# ax = sns.violinplot(x='experiment', y='fitness', data=df)

df1 = df.groupby('experiment')
df1.describe()

fitness                                                    \
               count      mean       std       min       25%       50%   
experiment                                                               
ANN Direct   85104.0  9.350364  1.022872  0.197193  9.520000  9.586667   
ANN Twist   101240.0  9.483150  0.803493  0.491311  9.546667  9.603333   
FSM          68540.0  9.662061  0.018770  9.400000  9.656667  9.663333   

                                   value                                     \
                 75%       max     count      mean       std  min       25%   
experiment                                                                    
ANN Direct  9.633333  9.680000   85104.0  7.475097  4.487132  0.0  2.995768   
ANN Twist   9.646667  9.703333  101240.0  7.263648  4.137853  0.0  2.997210   
FSM         9.670000  9.706500   68540.0  7.448548  3.791744  0.0  2.999790   

                                      
            50%       75%        max  
experiment                            
ANN Direct  7.5  10.95177  16.000000  
ANN Twist   7.5  10.32969  16.000000  
FSM         7.5  10.45119  15.999997

In [ ]:
fsm_replicates_combined = pd.concat(fsm_replicates).reset_index(drop=True)
bnn_replicates_combined = pd.concat(bnn_replicates).reset_index(drop=True)

In [ ]:
df_fsm_best = fsm_replicates_combined.nlargest(240, 'fitness').copy()
df_bnn_best = bnn_replicates_combined.nlargest(240, 'fitness').copy()
df_fsm_best.describe()

In [ ]:
# Physical Characteristics

col_names = ['fitness', 'Wheel Base', 'Track Width', 'Wheel Radius', 'Weg Count']

df_fsm_phys = df_fsm_best[col_names].copy()
df_bnn_phys = df_bnn_best[col_names].copy()
df_fsm_phys['Category'] = 'FSM'
df_bnn_phys['Category'] = 'ANN'
df = pd.concat([df_fsm_phys, df_bnn_phys], keys=['FSM', 'ANN'], axis=0)
df['all'] = ''


col_offset = 5
col_count = 4
col_names = df_fsm_best.columns[col_offset:col_offset + col_count]
ncols = len(col_names) + 1
fsize = ncols * 2.5, 9/2
fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=fsize)


for ax, col_name, map_idx in zip(axes, ['fitness'] + list(col_names), range(-1, ncols)):
    if map_idx == -1:
        ymin, ymax = 0, 10
        fancy_name = 'Fitness'
    else:
        m = fsm_map[map_idx]
        ymin, ymax = m['T'](m['minval']), m['T'](m['maxval'])
        fancy_name = m['fancy_name']
    
    # Plot the violin for this parameter
    sns.violinplot(x='all'
                   , y=col_name
                   , data=df
                   , hue='Category'
                   , ax=ax
                   , inner=None
                   , split=True
                   , scale_hue=False
                   , scale='width'
                  )

    # Set the y limits with some padding
    y_padding = (ymax - ymin) * 0.1
    ylims_with_padding = [ymin - y_padding, ymax + y_padding]
    ax.set_ylim(ylims_with_padding)
    
    # Prettify the lines and ticks
    if col_name == 'Weg Count':
        yticks = np.linspace(ymin, ymax, num=ymax + 1)
    else:
        yticks = np.linspace(ymin, ymax, num=5)
    ax.set_yticks(yticks)
    ax.set_yticklabels([round(yval, 1) for yval in yticks], fontsize=14)
    ax.set_xlabel(fancy_name, fontsize=18)
    ax.set_ylabel('')
    
    ax.legend_.remove()
    ax.plot([0, 0], ylims_with_padding, color='0.3', linestyle='-', linewidth=3)
    
sns.despine(top=True, right=True, left=True, bottom=True)
plt.tight_layout()

lgd = axes[-1].legend(loc=(1,0.5), fontsize=16)

ttl = fig.suptitle('Evolved Fitness and Physical Characteristics (Without Obstacles)', 
                   fontsize=24, y=1.1)

# fig.savefig('morph-params.png')
# handles, labels = axes[-1].get_legend_handles_labels()
# lgd = axes[-1].legend(handles, labels, loc=(1, 0.5), fontsize=16)#, bbox_to_anchor=(0.5,-0.1))

# fig.savefig('params1.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
# fig.savefig('params2.png', bbox_extra_artists=(lgd,), bbox_inches='tight')


In [ ]:
fig.savefig('../paper/figures/4-results/0-1-best_params.png', 
            bbox_extra_artists=(lgd, ttl), bbox_inches='tight')

In [ ]:
def plot_violins(df, col_names, map_offset, violin_size, exp_map):
    col_count = len(col_names)
    df_violin = df[col_names].copy()
    
    figsize = violin_size[0] * col_count, violin_size[1]

    fig, axes = plt.subplots(nrows=1, ncols=col_count, figsize=figsize)

    for ax, col_name, map_idx in zip(axes, col_names, range(map_offset, map_offset + col_count)):
        m = exp_map[map_idx]
        ymin, ymax = m['T'](m['minval']), m['T'](m['maxval'])

        # Plot the violin for this parameter
        sns.violinplot(df_violin[col_name], ax=ax, orient='v')

        # Set the y limits with some padding
        y_padding = (ymax - ymin) * 0.1
        ylims_with_padding = [ymin - y_padding, ymax + y_padding]
        ax.set_ylim(ylims_with_padding)

        # Prettify the lines and ticks
        yticks = np.linspace(ymin, ymax, num=5)
        ax.set_yticks(yticks)
        ax.set_yticklabels([round(yval, 1) for yval in yticks], fontsize=14)
        ax.set_xlabel(m['fancy_name'], fontsize=18)
        ax.set_ylabel('')

    sns.despine(top=True, right=True, left=True, bottom=True)
    plt.tight_layout()
    return fig, axes

In [ ]:
col_start = 9
col_count = 11
figsize = (2.5, 4.5)
col_names = list(df_fsm_best.columns[col_start:col_start + col_count])
fig, axes = plot_violins(df_fsm_best, col_names, col_start - 5, figsize, fsm_map)
axes[4].invert_yaxis()
axes[10].invert_yaxis()

ttl = fig.suptitle('Evolved FSM-0-1 Parameters', fontsize=24, y=1.1)

In [ ]:
fig.savefig('../paper/figures/4-results/FSM-0-1-best_params.png', 
            bbox_extra_artists=(ttl,), bbox_inches='tight')

In [ ]:
col_start = 14
col_count = 6
col_names = list(df_fsm_best.columns[col_start:col_start + col_count])
fig, axes = plot_violins(df_fsm_best, col_names, col_start - 5, figsize, fsm_map)
axes[5].invert_yaxis()

In [ ]:
fig.savefig('../paper/figures/4-results/FSM-0-1-best_params2.png', 
            bbox_extra_artists=(ttl,), bbox_inches='tight')

In [ ]:
col_start = 10
col_count = 6
col_names = list(df_bnn_best.columns[col_start:col_start + col_count])
fig1, axes1 = plot_violins(df_bnn_best, col_names, col_start - 5, figsize, bnn_map)

col_start = 16
col_count = 6
col_names = list(df_bnn_best.columns[col_start:col_start + col_count])
fig2, axes2 = plot_violins(df_bnn_best, col_names, col_start - 5, figsize, bnn_map)

In [ ]:
df_fsm_best.describe()